<a href="https://colab.research.google.com/github/mswastik/forecasting/blob/main/SARIMAv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install pmdarima

In [30]:
import pmdarima as pmd
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf,pacf,adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tools.eval_measures import rmse
from sklearn.model_selection import TimeSeriesSplit,cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np
import warnings
import plotly.express as px
import cufflinks as cf
import plotly.tools as tls

In [31]:
!git clone -l -s https://github.com/mswastik/forecasting.git cloned-repo
%cd cloned-repo

Cloning into 'cloned-repo'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 30 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (30/30), done.
/content/cloned-repo/cloned-repo


In [32]:
warnings.filterwarnings("ignore")
df = pd.read_excel('Forecasting1.xlsx',sheet_name='SO',index_col='del date')
df.rename({'agg':'agg1'},axis=1,inplace=True)
df['dd']=df.index.copy()
df['month'] = df['dd'].dt.year.astype('str')+"-"+df['dd'].dt.month.astype('str')
df2=df.groupby(['month','agg1']).sum().copy()
df2.drop(['wt','cs size'],inplace=True,axis=1)

In [40]:
toot=df.loc[df.agg1=='Tmmthpkhti Limi Yih'].copy()
toot=pd.DataFrame(toot['cases'])
toot=toot.resample('M').sum()

In [ ]:
cint=inv_boxcox(mod.get_forecast(6).conf_int(),fitted_lambda)-1

In [ ]:
df2=df2.join(ff,how="outer")

ValueError: columns overlap but no suffix specified: Index(['pred'], dtype='object')

In [ ]:
df2[['Qty','pred']]

,Qty,pred
2012-12-31,1148.0,NaN
2013-01-31,997.0,NaN
2013-02-28,560.0,NaN
2013-03-31,560.0,NaN
2013-04-30,1204.0,NaN
...,...,...
2020-08-31,NaN,1001.931909
2020-09-30,NaN,970.645824
2020-10-31,NaN,981.301929
2020-11-30,NaN,977.644404


In [ ]:
ff

,pred
2014-12-31,599.925527
2015-01-31,513.593621
2015-02-28,596.156350
2015-03-31,535.449562
2015-04-30,830.416815
...,...
2020-08-31,1001.931909
2020-09-30,970.645824
2020-10-31,981.301929
2020-11-30,977.644404


In [ ]:
import plotly.io as pio
pio.renderers.default = "colab"

In [ ]:
px.line(toot,y='cases',x=toot.index)

In [ ]:
px.box(toot,y='cases')

In [41]:
toot=toot.loc[toot.cases<=80000]

In [ ]:
adf=pmd.arima.ADFTest(alpha=.05)
adf.should_diff(toot[['cases']])

(0.49644081219960445, True)

In [ ]:
from pmdarima.arima import ndiffs
ndiffs(toot, alpha=0.05, test='kpss', max_d=6)

1

In [ ]:
def adf_test(timeseries):
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
adf_test(toot)

Results of Dickey-Fuller Test:
Test Statistic                 -1.755483
p-value                         0.402776
#Lags Used                      6.000000
Number of Observations Used    88.000000
Critical Value (1%)            -3.506944
Critical Value (5%)            -2.894990
Critical Value (10%)           -2.584615
dtype: float64


In [42]:
toot = toot.diff(1)
toot.dropna(inplace=True)

In [43]:
model1 = pmd.arima.auto_arima(toot['cases'], start_p=1, start_q=1,start_P=1, start_Q=1, m=12,
                     max_p=6, max_q=6, max_P=6, max_Q=6, seasonal=True,stepwise=True)
model1

ARIMA(maxiter=50, method='lbfgs', order=(0, 0, 1), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(2, 0, 0, 12),
      start_params=None, suppress_warnings=False, trend=None,
      with_intercept=False)

In [62]:
n_train = 24
n_records = len(toot)
res=[]
for i in range(n_train, n_records):
    pre=[]
    train, test = toot[0:i], toot[i:i+1]
    mod = SARIMAX(train, order=(0, 0, 1), seasonal_order=(2, 0, 0, 3))
    mod=mod.fit()
    fore.update(mod.forecast())
    pre = mod.forecast()
    res.extend(rmse(test,pre))
#df1.plot(legend=True,figsize=(15,10))
#pre.plot(legend=True)

In [66]:
kk=mod.plot_diagnostics()
kk.set_size_inches(15,13)
tls.mpl_to_plotly(kk)

In [63]:
res

[6999.033134437161,
 10759.11743113925,
 93.59936024221861,
 3555.601574993987,
 8676.303262055142,
 979.2895483560951,
 1519.2984916720443,
 5365.140952795194,
 6511.944322658441,
 7148.698685486474,
 4916.128382487596,
 4923.71580081229,
 2787.273769066402,
 2038.8909667953121,
 3101.8453978421594,
 5166.510260555782,
 1236.2224101721586,
 4807.663331296663,
 2178.189783794028,
 6731.619250165473,
 5464.171871421065,
 9775.41784497316,
 4820.955982169347,
 3641.4869252217377,
 627.0511253300897,
 4477.615657303447,
 1995.5740431127429,
 2612.3340499840515,
 4967.280023483081,
 7258.729994109863,
 2664.190623060521,
 29900.490826868565,
 6343.293076256126,
 15396.589294631316,
 15241.094821057459,
 41686.660981056,
 15384.063974342404,
 16401.057593876096,
 9615.170372233704,
 11876.614235806404,
 32613.831561361396,
 2431.1250727563856,
 7701.0761647771615,
 1373.0349158083545,
 39283.631394213204,
 592.5911231303035,
 3902.6361766845257,
 45449.288080732236,
 12426.233042491578,
 12

In [64]:
print("Avg rmse is %d" %np.mean(res))

Avg rmse is 11204


In [65]:
mod.get_forecast(6).conf_int()

,lower cases,upper cases
93,-15699.988162,41292.435638
94,-46756.966589,16665.465272
95,-29071.925267,34350.506594
96,-33558.716445,32006.382284
97,-21968.040161,44097.556789
98,-33953.215027,32112.381924


In [ ]:
from scipy.stats import boxcox
from scipy.special import inv_boxcox
df3,fitted_lambda= boxcox(df1['Qty']+1,lmbda=None)

NameError: ignored

In [ ]:
def evaluate_arima_model(X, arima_order,s_order):
    splits = TimeSeriesSplit(n_splits=3)
    error=[]
    # make predictions
    for train_index, test_index in splits.split(X):
        train = X.values[train_index]
        test = X.values[test_index]
        model = SARIMAX(train, order=arima_order,seasonal_order=s_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
        error.append(rmse(test.T[0], yhat))
    return np.asarray(error).mean()
 
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values, m):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                for P in p_values:
                    for D in d_values:
                        for Q in q_values:
                            order = (p,d,q)
                            s_order = (P,D,Q,m)
                            try:
                                mse = evaluate_arima_model(dataset, order,s_order)
                                if mse < best_score:
                                    best_score, best_cfg = mse, (order,s_order)
                                print('SARIMA%sx%s RMSE=%.3f' % (order,s_order,mse))
                            except:
                                continue
    print('Best SARIMA %s RMSE=%.3f' % (best_cfg, best_score))

In [ ]:
p_values = [0,1,2,3,4]
d_values = range(0, 2)
q_values = range(0, 4)
evaluate_models(df1, p_values, d_values, q_values,12)

SARIMA(0, 0, 0)x(0, 0, 0, 12) RMSE=1716.003
SARIMA(0, 0, 0)x(0, 0, 1, 12) RMSE=1554.912
SARIMA(0, 0, 0)x(0, 0, 2, 12) RMSE=1437.645
SARIMA(0, 0, 0)x(0, 0, 3, 12) RMSE=1374.687
SARIMA(0, 0, 0)x(0, 1, 0, 12) RMSE=1177.831
SARIMA(0, 0, 0)x(0, 1, 1, 12) RMSE=1152.103
SARIMA(0, 0, 0)x(0, 1, 2, 12) RMSE=1182.464
SARIMA(0, 0, 0)x(0, 1, 3, 12) RMSE=1179.829
SARIMA(0, 0, 0)x(1, 0, 0, 12) RMSE=1241.914
SARIMA(0, 0, 0)x(1, 0, 1, 12) RMSE=1274.391
SARIMA(0, 0, 0)x(1, 0, 2, 12) RMSE=1312.642
SARIMA(0, 0, 0)x(1, 0, 3, 12) RMSE=1308.297
SARIMA(0, 0, 0)x(1, 1, 0, 12) RMSE=1181.554
SARIMA(0, 0, 0)x(1, 1, 1, 12) RMSE=1197.323
SARIMA(0, 0, 0)x(1, 1, 2, 12) RMSE=1199.893
SARIMA(0, 0, 0)x(1, 1, 3, 12) RMSE=1179.862
SARIMA(0, 0, 0)x(2, 0, 0, 12) RMSE=1341.675
SARIMA(0, 0, 0)x(2, 0, 1, 12) RMSE=1317.626
SARIMA(0, 0, 0)x(2, 0, 2, 12) RMSE=1327.595
SARIMA(0, 0, 0)x(2, 0, 3, 12) RMSE=1325.177
SARIMA(0, 0, 0)x(2, 1, 0, 12) RMSE=1192.402
SARIMA(0, 0, 0)x(2, 1, 1, 12) RMSE=1184.396
SARIMA(0, 0, 0)x(2, 1, 2, 12) RM

In [ ]:
ff = pd.DataFrame.from_dict(data=fore,orient='index',columns=["pred"])
pre1 = inv_boxcox(mod.get_forecast(6).predicted_mean,fitted_lambda)-1
pre1=pd.DataFrame(pre1)
pre1.rename(columns={0:"pred"},inplace=True)
ff=ff.append(pre1)

In [ ]:
import plotly.graph_objects as go
fig=px.line(data_frame=df2,x=df2.index,y='Qty')
fig.add_scatter(x=cint.index,y=cint['lower y'])
fig.add_scatter(x=cint.index,y=cint['upper y'],fill='tonexty')
fig.add_scatter(x=ff.index,y=ff['pred'])
#fig2=px.area(cint)
#fig.add_trace()

In [ ]:
fig2=px.area(cint,x=cint.index,y='lower y')
fig2.add_scatter(x=cint.index,y=cint['upper y'])

In [ ]:
ndiffs(df1, alpha=0.05, test='adf', max_d=6)

1

In [ ]:
adf_test(df2.dropna())

Results of Dickey-Fuller Test:
Test Statistic                -6.621722e+00
p-value                        6.013716e-09
#Lags Used                     4.000000e+00
Number of Observations Used    8.600000e+01
Critical Value (1%)           -3.508783e+00
Critical Value (5%)           -2.895784e+00
Critical Value (10%)          -2.585038e+00
dtype: float64


In [68]:
#plt.figure(figsize=(15,10))
#from plotly.plotly import plot_mpl
hh=seasonal_decompose(x=toot,model='additive').plot()
hh.set_size_inches(15,13)
tls.mpl_to_plotly(hh)

ValueError: ignored

In [ ]:
cf.set_config_file(offline_show_link=False)